# Rosey Training Notebook 

Basic supervised neural network

In [ ]:
import sys
# print(sys.path)
sys.path.append('/home/michael/Github/ai_training_notebooks/standard_utils')
from general_ai_training_utils import BaseSupervisedTrainer
import rosey

### Instantiate a base supervised learner with the Rosey model 

In [ ]:
model = rosey.Rosey("utils")
model_trainer = BaseSupervisedTrainer(export_directory="/home/michael/Desktop", model=model, utils_file="utils", data_directory="/media/michael/BigMemory/datasets/jetsoncar/backup1-12-20/data")

In [ ]:
model_trainer.set_dataset()

In [ ]:
model_trainer.load_dataset(dynamic=True, percent_training=0.8, existing_npy_directory=None,  npy_save_directory=None, total_npy_size=100, num_stacked_images=1)
print("data loaded") # Remove this line to see returned values for training and steering (include test values if dynamic load)

In [ ]:
from keras.optimizers import Adam
model_trainer.build_model(loss='mean_squared_error', optimizer=Adam(1.0e-4), regularizer=0.01)

In [ ]:
model_trainer.train_model(batch_size=40, validation_steps=1000, nb_epochs=10, steps_per_epoch=1500)

## Update config.yaml and notes.txt to properly document and specify the opartion and development of the model

### notes.txt
Here you should put notes about model performance, test conditions, and todo list for improving the model. 

### config.yaml
Specify the maximum update rate for the model when run on the jetson TX2

In [ ]:
# preapare desired model file for export
import os
model_to_export = "model.009-2.58.h5"
os.rename(model_to_export, 'model.h5')

In [ ]:
# copy the relevant files and export model
model_name = "RoseyV2"
if model_trainer.export_model("/home/michael/Desktop/" + model_name):
    print("Model export successful!")